# 1. identity extraction

In [ ]:
import os
import argparse
import numpy as np 
from tqdm import  tqdm
from glob import glob
from embedding_distribution import load_image, get_frame_id, frames_per_video
from config.config import Config
from torch.nn import DataParallel
from models import resnet_face18
import torch
import cv2
torch.cuda.empty_cache()

test_model_path= "./pretrained/resnet18_110.pth"
opt = Config()
model = resnet_face18(opt.use_se)
model = DataParallel(model)
model.load_state_dict(torch.load(test_model_path))
model.to(torch.device("cuda"))
model.eval()

def video_id_extraction(path, 
                        save, 
                        len_save, 
                        batch_size = 16,
                        compress=None, 
                        noise=None):
    whole_set_ids = None
    video_length = []
    video_list = glob(path+"/*")
    video_list.sort()
    for video in tqdm(video_list):
        batch_frames = None
        video_ids = None
        batch_count = 0
        video_frames = glob(video+"/*")
        if len(glob(video+"/*")) < 1:
          print("empty folder", video)
          continue
        frame_list = glob(video+"/*")
        for frame in frame_list:
            batch_count += 1
            try:
             single_image = load_image(frame, compress=compress, noise=noise)
            except Exception as e:
              print("blocked-", frame)
              print("error-",e)
              del e
              continue
            if batch_frames is None:
                batch_frames = single_image
            else:
                batch_frames = np.concatenate((batch_frames, single_image), axis=0)
            if not batch_count % batch_size or batch_count==len(video_frames):
                batch_ids = get_frame_id(model, batch_frames, batch_size=batch_size)
                batch_frames = None
                if video_ids is None:
                    video_ids = batch_ids
                else:
                    video_len1 = video_ids.shape[0]
                    video_ids = np.concatenate((video_ids, batch_ids), axis=0)
        if whole_set_ids is None:
            whole_set_ids = video_ids
        else:
            whole_set_ids = np.concatenate((whole_set_ids, video_ids), axis=0)
        video_length.append(len(video_frames))
    video_length = np.array(video_length) 
    print("video_length", sum(video_length))
    print("whole_set_ids", len(whole_set_ids))
    np.save(save, whole_set_ids)
    np.save(len_save,video_length)
    del whole_set_ids, video_length

### 1.1. FaceForensics++  dataset

In [ ]:
save_root = "./id_vectors/FFpp/"

paths=["/home/baoping/FaceForensics++/manipulated_sequences/NeuralTextures/raw/aligned_faces", \
       "/home/baoping/FaceForensics++/manipulated_sequences/FaceSwap/raw/aligned_faces", \
       "/home/baoping/FaceForensics++/manipulated_sequences/Face2Face/raw/aligned_faces", \
       "/home/baoping/FaceForensics++/manipulated_sequences/Deepfakes/raw/aligned_faces", \
       "/home/baoping/FaceForensics++/original_sequences/youtube/raw/aligned_faces"  ]

ids_save_path = ["ffpp_nt_ids.npy", "ffpp_fs_ids.npy", "ffpp_ff_ids.npy", "ffpp_df_ids.npy", "ffpp_real_ids.npy"]#
len_save_path = ["ffpp_nt_lens.npy", "ffpp_fs_lens.npy", "ffpp_ff_lens.npy", "ffpp_df_lens.npy", "ffpp_real_lens.npy"]#
for idx, p in enumerate(paths):
    video_id_extraction(p, 
                        os.path.join( ids_save_path[idx] ), 
                        os.path.join( len_save_path[idx]) )

In [ ]:
import numpy as np
import os

def set_merge(fake_id_list, real_length):
    fake_type_count = len(fake_id_list)
    real_video_count = real_length
    fake_video_count = int(real_video_count // fake_type_count)
    print("video count", real_video_count, fake_video_count)
    ret_list = []
    for i in fake_id_list:
        ret_list += i[:fake_video_count]
    return ret_list
ffpp_real_ids = "./id_vectors/FFpp/ffpp_real_ids.npy"
ffpp_real_len = "./id_vectors/FFpp/ffpp_real_lens.npy"
ffpp_df_ids = "./id_vectors/FFpp/ffpp_df_ids.npy"
ffpp_df_len = "./id_vectors/FFpp/ffpp_df_lens.npy"
ffpp_ff_ids = "./id_vectors/FFpp/ffpp_ff_ids.npy"
ffpp_ff_len = "./id_vectors/FFpp/ffpp_ff_lens.npy"
ffpp_fs_ids = "./id_vectors/FFpp/ffpp_fs_ids.npy"
ffpp_fs_len = "./id_vectors/FFpp/ffpp_fs_lens.npy"
ffpp_nt_ids = "./id_vectors/FFpp/ffpp_nt_ids.npy"
ffpp_nt_len = "./id_vectors/FFpp/ffpp_nt_lens.npy"

fake_ids_list = [ffpp_df_ids, ffpp_ff_ids, ffpp_fs_ids, ffpp_nt_ids]
fake_len_list = [ffpp_df_len, ffpp_ff_len, ffpp_fs_len, ffpp_nt_len]

real_ids = np.load(ffpp_real_ids)
real_len = np.load(ffpp_real_len, allow_pickle=True)
print("real shapes", real_ids.shape, real_len.shape)

merged_ids = None
merged_len = None
for i,l in zip(fake_ids_list,fake_len_list):#range(4):
    fake_ids = np.load(i)
    fake_len = np.load(l, allow_pickle=True)
    if merged_ids is None:
        merged_ids = fake_ids
        merged_len = fake_len
    else:
        merged_ids = np.concatenate((merged_ids,fake_ids))
        merged_len = np.concatenate((merged_len,fake_len))
np.save("./id_vectors/FFpp/ffpp_fake_ids.npy", merged_ids)
np.save("./id_vectors/FFpp/ffpp_fake_len.npy", merged_len)

### 1.2. CelebDF

In [ ]:
save_root = "./id_vectors/CelebDF/"

paths=["/media/baoping/8TSGHDD/CelebDF/Celeb-real/aligned_faces", \
       "/media/baoping/8TSGHDD/CelebDF/Celeb-synthesis/aligned_faces"]

ids_save_path = ["celebdf_real.npy", "celebdf_fake.npy"]
len_save_path = ["celebdf_real_lens.npy", "celebdf_fake_lens.npy"]
for idx, p in enumerate(paths):
    video_id_extraction(p, 
                        os.path.join(save_root, ids_save_path[idx]), 
                        os.path.join(save_root, len_save_path[idx]))

### 1.3. DFD

In [ ]:
save_root = "./id_vectors/DFD/"

paths=["/media/baoping/8TSGHDD/DeepFakeDetection/real", \
       "/media/baoping/8TSGHDD/DeepFakeDetection/fake"]

ids_save_path = ["dfd_real.npy", "dfd_fake.npy"]
len_save_path = ["dfd_real_lens.npy", "dfd_fake_lens.npy"]
for idx, p in enumerate(paths):
    video_id_extraction(p, 
                        os.path.join(save_root, ids_save_path[idx]), 
                        os.path.join(save_root, len_save_path[idx]))

### 1.4.Deeper

In [ ]:
save_root = "./id_vectors/Deeper/"

paths=["/media/baoping/8TSGHDD/DeeperForensics1.0/manipulated_videos/end_to_end"]

ids_save_path = ["deeper_fake.npy"]
len_save_path = ["deeper_fake_lens.npy"]
for idx, p in enumerate(paths):
    video_id_extraction(p, 
                        os.path.join(save_root, ids_save_path[idx]), 
                        os.path.join(save_root, len_save_path[idx]))

# 2. TI2Net training

In [14]:
!python train.py \
--fake_selection="all" \
--model="test" \
--train_contrastive=1 \
--real_path="./id_vectors/FFpp/ffpp_real_ids.npy" \
--real_len_path="./id_vectors/FFpp/ffpp_real_lens.npy" \
--fake_path="./id_vectors/FFpp/ffpp_fake_ids.npy" \
--fake_len_path="./id_vectors/FFpp/ffpp_fake_lens.npy"

Using device: cuda:0
train set and test and set get
Training contrastive module
model TI2Net(
  (dropout_landmark): LandmarkDropout()
  (rnn): GRU(1024, 512, batch_first=True, bidirectional=True)
  (dropout_feature): Dropout(p=0.7, inplace=False)
  (linear_1): Linear(in_features=1024, out_features=128, bias=True)
  (relu): ReLU()
  (linear_2): Linear(in_features=128, out_features=2, bias=True)
  (output): Softmax(dim=1)
)
Using TripletMarginLoss
train iteration each epoch: 2997
test iteration each epoch: 999
***************************************************************************************************
epoch:1 training: emb_loss:0.3214 clf_loss:0.6032, loss:0.6353, train_acc:0.6607save the model to ./weights/torch/test.pth
epoch:1 testing: test_seq_acc:0.5315, test_seq_auc:0.7059, test_video_acc:0.5315, test_video_auc:0.7059save the model to ./weights/torch/test.pth
epoch:1 summary: best_seq_acc:0.5315 best_seq_auc:0.7059 best_video_acc:0.5315 best_video_auc:0.7059save the model to

***************************************************************************************************
epoch:14 training: emb_loss:0.07202 clf_loss:0.4432, loss:0.4504, train_acc:0.8638save the model to ./weights/torch/test.pth
epoch:14 testing: test_seq_acc:0.6722, test_seq_auc:0.891, test_video_acc:0.6722, test_video_auc:0.891save the model to ./weights/torch/test.pth
epoch:14 summary: best_seq_acc:0.6722 best_seq_auc:0.8921 best_video_acc:0.6722 best_video_auc:0.8921save the model to ./weights/torch/test.pth
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:15 training: emb_loss:0.07112 clf_loss:0.4427, loss:0.4498, train_acc:0.8632save the model to ./weights/torch/test.pth
epoch:15 testing: test_seq_acc:0.7032, test_seq_auc:0.9107, test_video_acc:0.70

***************************************************************************************************
epoch:28 training: emb_loss:0.03811 clf_loss:0.405, loss:0.4088, train_acc:0.9042save the model to ./weights/torch/test.pth
epoch:28 testing: test_seq_acc:0.9014, test_seq_auc:0.9519, test_video_acc:0.9014, test_video_auc:0.9519save the model to ./weights/torch/test.pth
epoch:28 summary: best_seq_acc:0.9014 best_seq_auc:0.9527 best_video_acc:0.9014 best_video_auc:0.9527save the model to ./weights/torch/test.pth
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:29 training: emb_loss:0.03629 clf_loss:0.4006, loss:0.4043, train_acc:0.9088
epoch:29 testing: test_seq_acc:0.8764, test_seq_auc:0.934, test_video_acc:0.8764, test_video_auc:0.934
epoch:29 summary:

***************************************************************************************************
epoch:42 training: emb_loss:0.02134 clf_loss:0.3794, loss:0.3816, train_acc:0.9308
epoch:42 testing: test_seq_acc:0.8043, test_seq_auc:0.9577, test_video_acc:0.8043, test_video_auc:0.9577
epoch:42 summary: best_seq_acc:0.9219 best_seq_auc:0.9577 best_video_acc:0.9219 best_video_auc:0.9577
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:43 training: emb_loss:0.02493 clf_loss:0.3781, loss:0.3806, train_acc:0.9315
epoch:43 testing: test_seq_acc:0.9124, test_seq_auc:0.9493, test_video_acc:0.9124, test_video_auc:0.9493
epoch:43 summary: best_seq_acc:0.9219 best_seq_auc:0.9577 best_video_acc:0.9219 best_video_auc:0.9577
**************************************

***************************************************************************************************
epoch:56 training: emb_loss:0.01896 clf_loss:0.369, loss:0.3709, train_acc:0.9415
epoch:56 testing: test_seq_acc:0.8804, test_seq_auc:0.9615, test_video_acc:0.8804, test_video_auc:0.9615
epoch:56 summary: best_seq_acc:0.9505 best_seq_auc:0.9642 best_video_acc:0.9505 best_video_auc:0.9642
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:57 training: emb_loss:0.02237 clf_loss:0.3684, loss:0.3706, train_acc:0.9435
epoch:57 testing: test_seq_acc:0.9024, test_seq_auc:0.9531, test_video_acc:0.9024, test_video_auc:0.9531
epoch:57 summary: best_seq_acc:0.9505 best_seq_auc:0.9642 best_video_acc:0.9505 best_video_auc:0.9642
***************************************

***************************************************************************************************
epoch:71 training: emb_loss:0.01308 clf_loss:0.3593, loss:0.3606, train_acc:0.951
epoch:71 testing: test_seq_acc:0.7247, test_seq_auc:0.9608, test_video_acc:0.7247, test_video_auc:0.9608
epoch:71 summary: best_seq_acc:0.9505 best_seq_auc:0.9652 best_video_acc:0.9505 best_video_auc:0.9652
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:72 training: emb_loss:0.01322 clf_loss:0.3604, loss:0.3617, train_acc:0.9516
epoch:72 testing: test_seq_acc:0.8619, test_seq_auc:0.942, test_video_acc:0.8619, test_video_auc:0.942
epoch:72 summary: best_seq_acc:0.9505 best_seq_auc:0.9652 best_video_acc:0.9505 best_video_auc:0.9652
*****************************************

***************************************************************************************************
epoch:86 training: emb_loss:0.009915 clf_loss:0.3534, loss:0.3544, train_acc:0.958
epoch:86 testing: test_seq_acc:0.7973, test_seq_auc:0.9704, test_video_acc:0.7973, test_video_auc:0.9704
epoch:86 summary: best_seq_acc:0.9505 best_seq_auc:0.9747 best_video_acc:0.9505 best_video_auc:0.9747
***************************************************************************************************
                                                                                

***************************************************************************************************
epoch:87 training: emb_loss:0.008228 clf_loss:0.3522, loss:0.353, train_acc:0.9602
epoch:87 testing: test_seq_acc:0.8428, test_seq_auc:0.9677, test_video_acc:0.8428, test_video_auc:0.9677
epoch:87 summary: best_seq_acc:0.9505 best_seq_auc:0.9747 best_video_acc:0.9505 best_video_auc:0.9747
**************************************

# 3. TI2Net evaluation

In [11]:
!python evaluate.py \
--model="test.pth" \
--real_id_path="./id_vectors/FFpp/ffpp_real_ids_test.npy" \
--real_len_path="./id_vectors/FFpp/ffpp_real_lens_test.npy" \
--fake_id_path="./id_vectors/FFpp/ffpp_fake_ids_test.npy" \
--fake_len_path="./id_vectors/FFpp/ffpp_fake_lens_test.npy"

Using device: cuda:0
Please register the status.


#----Evaluation Results----#
Accuracy (sequence-level): 0.99
AUC (sequence-level): 0.998
Accuracy (video-level): 0.99
AUC (video-level): 0.998
#------------End-----------#
